## 定义取汉字函数

In [1]:
import re

# 提取汉字
def find_chinese(file):
    pattern = re.compile(r'[^\u4e00-\u9fa5]')
    chinese = re.sub(pattern, '', file)
    return chinese

# 提取非汉字字符
def find_unchinese(file):
    pattern = re.compile(r'[\u4e00-\u9fa5]')
    unchinese = re.sub(pattern,"",file)
    return unchinese


## 读取数据

In [2]:
import pandas as pd
data=pd.read_excel('data/复旦大学中文文本分类语料.xlsx','sheet1',encoding='utf-8') 

In [3]:
data.head()

,分类,正文
0,艺术,﻿【 文献号 】1-2432\n【原文出处】出版发行研究\n【原刊地名】京\n【原刊期号】1...
1,艺术,﻿【 文献号 】1-2435\n【原文出处】扬州师院学报：社科版\n【原刊期号】199504...
2,艺术,﻿【 文献号 】1-2785\n【原文出处】南通师专学报：社科版\n【原刊期号】199503...
3,艺术,﻿【 文献号 】1-3021\n【原文出处】社会科学战线\n【原刊地名】长春\n【原刊期号】...
4,艺术,﻿【 文献号 】1-3062\n【原文出处】上海文化\n【原刊期号】199505\n【原刊页...


## 去除文本中非汉字的内容

In [4]:
data['处理后的正文'] = data['正文'].apply(lambda x: find_chinese(x))

In [5]:
data.head()

,分类,正文,处理后的正文
0,艺术,﻿【 文献号 】1-2432\n【原文出处】出版发行研究\n【原刊地名】京\n【原刊期号】1...,文献号原文出处出版发行研究原刊地名京原刊期号原刊页号分类号分类名出版工作图书评介作者王益复印...
1,艺术,﻿【 文献号 】1-2435\n【原文出处】扬州师院学报：社科版\n【原刊期号】199504...,文献号原文出处扬州师院学报社科版原刊期号原刊页号分类号分类名出版工作图书评介作者王菊延复印期...
2,艺术,﻿【 文献号 】1-2785\n【原文出处】南通师专学报：社科版\n【原刊期号】199503...,文献号原文出处南通师专学报社科版原刊期号原刊页号分类号分类名语言文字学作者咏枫复印期号标题语...
3,艺术,﻿【 文献号 】1-3021\n【原文出处】社会科学战线\n【原刊地名】长春\n【原刊期号】...,文献号原文出处社会科学战线原刊地名长春原刊期号原刊页号分类号分类名文艺理论作者李心峰复印期号...
4,艺术,﻿【 文献号 】1-3062\n【原文出处】上海文化\n【原刊期号】199505\n【原刊页...,文献号原文出处上海文化原刊期号原刊页号分类号分类名文艺理论作者朱立元复印期号标题中西古代艺术...


## 分词

In [6]:
import jieba
jieba.enable_parallel(64) #并行分词开启
data['处理后的正文']  = data['处理后的正文'].apply(lambda i:jieba.cut(i))
data['处理后的正文']  = [' '.join(i) for i in data['处理后的正文']]

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.699 seconds.
Prefix dict has been built succesfully.


In [7]:
data.head()

,分类,正文,处理后的正文
0,艺术,﻿【 文献号 】1-2432\n【原文出处】出版发行研究\n【原刊地名】京\n【原刊期号】1...,文献号 原文 出处 出版发行 研究 原刊 地名 京原 刊期 号 原刊 页 号 分类号 分类 ...
1,艺术,﻿【 文献号 】1-2435\n【原文出处】扬州师院学报：社科版\n【原刊期号】199504...,文献号 原文 出处 扬州 师院 学报 社科 版原 刊期 号 原刊 页 号 分类号 分类 名 ...
2,艺术,﻿【 文献号 】1-2785\n【原文出处】南通师专学报：社科版\n【原刊期号】199503...,文献号 原文 出处 南通 师专 学报 社科 版原 刊期 号 原刊 页 号 分类号 分类 名 ...
3,艺术,﻿【 文献号 】1-3021\n【原文出处】社会科学战线\n【原刊地名】长春\n【原刊期号】...,文献号 原文 出处 社会科学 战线 原刊 地名 长春 原刊 期号 原刊 页 号 分类号 分类...
4,艺术,﻿【 文献号 】1-3062\n【原文出处】上海文化\n【原刊期号】199505\n【原刊页...,文献号 原文 出处 上海 文化 原刊 期号 原刊 页 号 分类号 分类 名 文艺理论 作者 ...


## 去除停止词

In [8]:
stop = [line.strip() for line in open('data/停用词汇总.txt','r',encoding='utf-8').readlines()]
data['处理后的正文'] = data['处理后的正文'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

## 保存文件

In [9]:
import os 
if not os.path.exists('./output'):
    os.mkdir('./output')

f_data = open('./output/data_clean_split.txt','w',encoding='utf-8')
for i in range (len(data)):
    line = data['分类'][i] + '\t' + data['处理后的正文'][i] + '\n'
    f_data.write(line)
print("写入成功.....")

f_data.close()

写入成功.....
